In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load image
path_to_image = '/data2/eranario/data/yolo_grl_data/BordenNight/Synthetic/images/00000.jpg'
image = cv2.imread(path_to_image)

# convert to rgb
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# display image
plt.imshow(image)

In [ ]:
# get center points of bounding box
path_to_label = '/data2/eranario/data/yolo_grl_data/BordenNight/Synthetic/labels/00000.txt'

# read label
with open(path_to_label, 'r') as f:
    label = f.readlines()
    
# get center points
center_points = []
for line in label:
    x_center, y_center, width, height = line.split(' ')[1:]
    center_points.append((x_center, y_center))
    
# convert center points to float and convert using image shape
center_points = [(float(x) * image.shape[1], float(y) * image.shape[0]) for x, y in center_points]

# plot on image
for x, y in center_points:
    cv2.circle(image, (int(x), int(y)), 5, (255, 0, 0), -1)
    
# crop image but keep all center points showing
cropped_image = image[int(min([y for x, y in center_points])):int(max([y for x, y in center_points])),
                        int(min([x for x, y in center_points])):int(max([x for x, y in center_points]))]
plt.imshow(cropped_image)

In [ ]:
# Create an empty attention map
attention_map = np.zeros(image.shape[:2])

# Convert center_points to a NumPy array
center_points = np.array(center_points)

# Create a grid of coordinates
x_coords = np.arange(image.shape[1])
y_coords = np.arange(image.shape[0])
x_grid, y_grid = np.meshgrid(x_coords, y_coords)

# Calculate the distance to each center point
for x, y in center_points:
    dist_squared = (x_grid - x) ** 2 + (y_grid - y) ** 2
    attention_map += np.exp(-dist_squared / (2 * 25 ** 2))

# Normalize the attention map
attention_map /= np.max(attention_map)

# Plot the attention map
plt.imshow(attention_map, cmap='viridis')

In [ ]:
# get example attention map for random set of center points of the same length
random_center_points = np.random.randint(0, image.shape[1], (len(center_points), 2))
random_attention_map = np.zeros(image.shape[:2])

# Convert center_points to a NumPy array
random_center_points = np.array(random_center_points)

# Calculate the distance to each center point
for x, y in random_center_points:
    dist_squared = (x_grid - x) ** 2 + (y_grid - y) ** 2
    random_attention_map += np.exp(-dist_squared / (2 * 50 ** 2))
    
# Normalize the attention map
random_attention_map /= np.max(random_attention_map)

# Plot the attention map
plt.imshow(random_attention_map, cmap='viridis')

In [ ]:
# FULL EXAMPLE

import cv2
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from tqdm import tqdm

# Load the image
path_to_image = '/data2/eranario/data/yolo_grl_data/CropGANData/detection_datasets/BordenDayRow/train/images/GH010070000300.jpg'
image = cv2.imread(path_to_image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

# Read label file to get center points
path_to_label = '/data2/eranario/data/yolo_grl_data/CropGANData/detection_datasets/BordenDayRow/train/labels/GH010070000300.txt'
with open(path_to_label, 'r') as f:
    label = f.readlines()

# Extract center points
center_points = []
for line in label:
    _, x_center, y_center, width, height = map(float, line.split())
    center_points.append((x_center * image.shape[1], y_center * image.shape[0]))

# Convert center points to a numpy array
center_points = np.array(center_points)

# Define the Gaussian map function
def gaussian_map(u, centers, sigma):
    attention_map = np.zeros((u.shape[0], u.shape[1]))
    for center in centers:
        dist_squared = (u[:, :, 0] - center[0]) ** 2 + (u[:, :, 1] - center[1]) ** 2
        attention_map += np.exp(-dist_squared / (2 * sigma ** 2))
    return attention_map

# Create a grid of coordinates
x_coords = np.arange(image.shape[1])
y_coords = np.arange(image.shape[0])
x_grid, y_grid = np.meshgrid(x_coords, y_coords)
grid = np.stack((x_grid, y_grid), axis=-1)

# Define the loss function to be minimized
def loss_function(centers, grid, target_map, sigma):
    centers = centers.reshape(-1, 2)
    current_map = gaussian_map(grid, centers, sigma)
    loss = np.sum((current_map - target_map) ** 2)
    return loss

# Create the target attention map using original center points
sigma = 20
target_attention_map = gaussian_map(grid, center_points, sigma)
target_attention_map /= np.max(target_attention_map)

# Initialize random center points for optimization
initial_centers = np.random.rand(len(center_points), 2)
initial_centers[:, 0] *= image.shape[1]
initial_centers[:, 1] *= image.shape[0]

# Generate the initial attention map using random center points
initial_attention_map = gaussian_map(grid, initial_centers, sigma)
initial_attention_map /= np.max(initial_attention_map)

# Flatten initial centers for minimize function
initial_centers_flat = initial_centers.flatten()

# Define a progress bar
progress_bar = tqdm(total=100, desc='Optimization Progress', unit='step')

# Callback function to update the progress bar
def callback(xk):
    progress_bar.update(1)

# Perform optimization with a progress bar
result = minimize(
    loss_function, initial_centers_flat, args=(grid, target_attention_map, sigma),
    method='L-BFGS-B', options={'maxiter': 100}, callback=callback
)

# Close the progress bar
progress_bar.close()

# Get optimized center points
optimized_centers = result.x.reshape(-1, 2)

# Generate attention map with optimized centers
optimized_attention_map = gaussian_map(grid, optimized_centers, sigma)
optimized_attention_map /= np.max(optimized_attention_map)

# Plot original image, initial, target, and optimized attention maps
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
ax[0].imshow(image)
ax[0].set_title('Original Image')
ax[1].imshow(initial_attention_map, cmap='viridis')
ax[1].set_title('Initial Attention Map')
ax[2].imshow(target_attention_map, cmap='viridis')
ax[2].set_title('Target Attention Map')
ax[3].imshow(optimized_attention_map, cmap='viridis')
ax[3].set_title('Optimized Attention Map')
plt.show()